# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction']


In [ ]:
import re
import json
import config as cfg
import swifter
import pandas as pd
from tqdm import tqdm
from pshmodule.utils import filemanager as fm

In [ ]:
data2 = []
with open('drive/MyDrive/Advertising_By_Personality/data/gpt/ext_gpt_mbti2.json', 'r', encoding='utf-8') as f:
    for line in f:
        data2.append(json.loads(line.rstrip('\n|\r')))
df_3000 = pd.DataFrame(data2)

In [ ]:
print(df_3000.columns)

Index(['mbti', 'marketing_entity', 'marketing_target', 'benefit_conditions',
       'benefits', 'discount_figure', 'promotional_items', 'promotional_place',
       'event_period', 'dow_information', 'season_information',
       'solicitation_point', 'type', 'label'],
      dtype='object')


In [ ]:
print(df_3000.shape)

(3000, 14)


In [ ]:
df_3000.reset_index(inplace=True, drop=True)

In [ ]:
df_3000.label.head()

0    꿈은 없고요 그냥 놀고 싶습니다🤸\\이왕이면 재밌게 게임🎮하고 최대 2만P💰 받고 ...
1    거짓말 같은 할인🤥\\하지만 거짓말이 아닌 [피노키오전] 50% 특가⚡ [TAG1]...
2    오늘은 꼭 빕스 가야 할걸?🏃\\오/늘/종/료⏰ [빕스 폭립 원.쁠.원] 이벤트✨ ...
3    비스포크가 포인트를 쏜다쏜다쏜다!🔫\\[비스포크]에 관한 감동 사연 남기고 3만P💰...
4    딱 1천P💰만 들고 커몽!\\[백악기 공룡의 전설 탐험展] 40% 할인권💌이랑 바꿔...
Name: label, dtype: object

# Data Check

In [ ]:
df_3000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mbti                3000 non-null   object
 1   marketing_entity    3000 non-null   object
 2   marketing_target    3000 non-null   object
 3   benefit_conditions  3000 non-null   object
 4   benefits            3000 non-null   object
 5   discount_figure     3000 non-null   object
 6   promotional_items   3000 non-null   object
 7   promotional_place   3000 non-null   object
 8   event_period        3000 non-null   object
 9   dow_information     3000 non-null   object
 10  season_information  3000 non-null   object
 11  solicitation_point  3000 non-null   object
 12  type                3000 non-null   object
 13  label               3000 non-null   object
dtypes: object(14)
memory usage: 328.2+ KB


In [ ]:
df_3000.solicitation_point.value_counts().iloc[201:250]

CJmall 뷰티전문관 BEAUTY                     1
신세계 아울렛                                 1
삐약삐약 뼝아리 친구들                            1
상세 미기재                                  1
우리 집                                    1
감성 충만한 밤                                1
부산                                      1
도산공원 맛집 볼피노                             1
에스 팩토리                                  1
럭키드로우                                   1
상세 정보 없음                                1
수능 공부하는 학생들                             1
사막 피부                                   1
0NE위크                                   1
올리브영 매장                                 1
CJ 브랜드의 외식점                             1
문화타운                                    1
외식할 때                                   1
제주 빛의 벙커 : 반고흐                          1
집에서 편하게 관람 가능 (따로 명시되지는 않음)             1
드라마 속 배경을 실제로 체험하는 곳                    1
준호집 말고 우리집                              1
제주, 그대, 나의 뮤즈 전시                        1
CGV골드클래스                          

# Processing

각각 에러 데이터 제거 2번 작업

In [ ]:
df_3000.at[1988, 'marketing_entity'] = '없음'
df_3000.at[1988, 'benefit_conditions'] = '없음'
df_3000.at[1988, 'discount_figure'] = '없음'
df_3000.at[1988, 'promotional_place'] = '없음'
df_3000.at[1988, 'dow_information'] = '없음'
df_3000.at[1988, 'solicitation_point'] = '없음'

In [ ]:
df_3000['marketing_entity'] = df_3000.marketing_entity.apply(lambda x: x.replace('없음, ', '').replace('없음, TAG1', 'TAG1').replace('(불명)', '없음').replace('<br>', '').replace('없음 (스팸 판매 업체)', '스팸 판매 업체').replace(' (정보 제공 없음)', '').replace('/TAG1', '').replace('/', ', ').replace("'", ""))
df_3000['marketing_target'] = df_3000.marketing_target.apply(lambda x: x.replace('있음 (또 오실 거죠?)', '없음'). replace('/', ', ').replace('타겟 :', '없음').replace(' or ', ','))
df_3000['benefit_conditions'] = df_3000.benefit_conditions.apply(lambda x: x.replace('<br>', '').replace('혜택 지급 조건 :', '없음').replace(' or ', ', '))
df_3000['benefits'] = df_3000.benefits.apply(lambda x: x.replace('<br>', '').replace(' (상세 내용 미기재)', '').replace(' (할인 정보 없음)', ''))
df_3000['discount_figure'] = df_3000.discount_figure.apply(lambda x: x.replace('있으나 문장에서 언급되지 않음', '없음').replace('할인 정보 없음', '없음'))
df_3000['promotional_items'] = df_3000.promotional_items.apply(lambda x: x.replace('<br>', ''))
df_3000['promotional_place'] = df_3000.promotional_place.apply(lambda x: x.replace('(미기재)', ''))
df_3000['event_period'] = df_3000.event_period.apply(lambda x: x.replace('(안내 X)', '').replace('없음 (선착순 할인 이벤트 시작 시간은 11시)', '선착순 할인 이벤트 시작 시간은 11시').replace('이벤트 기간 :', '없음'))
df_3000['season_information'] = df_3000.season_information.apply(lambda x: x.replace('<br>', '').replace('시즌 정보 :', '없음'))
df_3000['solicitation_point'] = df_3000.solicitation_point.apply(lambda x: x.replace('(소구점 정보 없음)', '').replace('소구점 :', '없음'))

In [ ]:
df_3000['marketing_entity'] = df_3000.marketing_entity.apply(lambda x: x.replace('마케팅 주체 : ', '').replace('마케팅 주체: ', ''))
df_3000['marketing_target'] = df_3000.marketing_target.apply(lambda x: x.replace('타겟 : ', '').replace('타겟: ', ''))
df_3000['benefit_conditions'] = df_3000.benefit_conditions.apply(lambda x: x.replace('혜택 지급 조건 : ', '').replace('혜택 지급 조건: ', ''))
df_3000['benefits'] = df_3000.benefits.apply(lambda x: x.replace('혜택 : ', '').replace('혜택: ', ''))
df_3000['discount_figure'] = df_3000.discount_figure.apply(lambda x: x.replace('할인 수치 : ', '').replace('할인 수치: ', ''))
df_3000['promotional_items'] = df_3000.promotional_items.apply(lambda x: x.replace('프로모션 품목 : ', '').replace('프로모션 품목: ', ''))
df_3000['promotional_place'] = df_3000.promotional_place.apply(lambda x: x.replace('프로모션 장소 : ', '').replace('프로모션 장소: ', ''))
df_3000['event_period'] = df_3000.event_period.apply(lambda x: x.replace('이벤트 기간 : ', '').replace('이벤트 기간: ', ''))
df_3000['dow_information'] = df_3000.dow_information.apply(lambda x: x.replace('요일 정보 : ', '').replace('요일 정보: ', ''))
df_3000['season_information'] = df_3000.season_information.apply(lambda x: x.replace('시즌 정보 : ', '').replace('시즌 정보: ', ''))
df_3000['solicitation_point'] = df_3000.solicitation_point.apply(lambda x: x.replace('소구점 : ', '').replace('소구점: ', ''))

In [ ]:
df_3000['marketing_entity'] = df_3000.marketing_entity.apply(lambda x: x.replace('[', '').replace(']', '').replace('+', ',').replace('O', '0').replace(' / ',', ').rstrip(',').rstrip('，').rstrip('.').strip())
df_3000['marketing_target'] = df_3000.marketing_target.apply(lambda x: x.replace('[', '').replace(']', '').replace('+', ',').replace('O', '0').replace(' / ',', ').rstrip(',').rstrip('，').rstrip('.').strip())
df_3000['benefit_conditions'] = df_3000.benefit_conditions.apply(lambda x: x.replace('[', '').replace(']', '').replace('+', ',').replace('O', '0').replace(' / ',', ').rstrip(',').rstrip('，').rstrip('.').strip())
df_3000['benefits'] = df_3000.benefits.apply(lambda x: x.replace('[', '').replace(']', '').replace('+', ',').replace('O', '0').replace(' / ',', ').rstrip(',').rstrip('，').rstrip('.').strip())
df_3000['discount_figure'] = df_3000.discount_figure.apply(lambda x: x.replace('[', '').replace(']', '').replace('+', ',').replace('O', '0').replace(' / ',', ').rstrip(',').rstrip('，').rstrip('.').strip())
df_3000['promotional_items'] = df_3000.promotional_items.apply(lambda x: x.replace('[', '').replace(']', '').replace('+', ',').replace('O', '0').replace(' / ',', ').rstrip(',').rstrip('，').rstrip('.').strip())
df_3000['promotional_place'] = df_3000.promotional_place.apply(lambda x: x.replace('[', '').replace(']', '').replace('+', ',').replace('O', '0').replace(' / ',', ').rstrip(',').rstrip('，').rstrip('.').strip())
df_3000['event_period'] = df_3000.event_period.apply(lambda x: x.replace('[', '').replace(']', '').replace('+', ',').replace('O', '0').replace(' / ',', ').rstrip(',').rstrip('，').rstrip('.').strip())
df_3000['dow_information'] = df_3000.dow_information.apply(lambda x: x.replace('[', '').replace(']', '').replace('+', ',').replace('O', '0').replace(' / ',', ').rstrip(',').rstrip('，').rstrip('.').strip())
df_3000['season_information'] = df_3000.season_information.apply(lambda x: x.replace('[', '').replace(']', '').replace('+', ',').replace('O', '0').replace(' / ',', ').rstrip(',').rstrip('，').rstrip('.').strip())
df_3000['solicitation_point'] = df_3000.solicitation_point.apply(lambda x: x.replace('[', '').replace(']', '').replace('+', ',').replace('O', '0').replace(' / ',', ').rstrip(',').rstrip('，').rstrip('.').strip())
df_3000['label'] = df_3000.label.apply(lambda x: x.replace('O', '0').strip())

In [ ]:
df_3000.fillna('없음', inplace=True)

### Save

In [ ]:
temp_dict = [{"marketing_entity": row['marketing_entity'], "marketing_target": row['marketing_target'], "benefit_conditions": row['benefit_conditions'], "benefits": row['benefits'], "discount_figure": row['discount_figure'], "promotional_items": row['promotional_items'], "promotional_place": row['promotional_place'], "event_period": row['event_period'], "dow_information": row['dow_information'], "season_information": row['season_information'], "solicitation_point": row['solicitation_point'], "type": row['type'], "label": row['label']} for _, row in df.iterrows()]
with open("drive/MyDrive/Advertising_By_Personality/data/gpt/train_ntnf2.json", 'w', encoding='utf-8') as f:
    for line in temp_dict:
        json_record = json.dumps(line, ensure_ascii=False)
        f.write(json_record + '\n')

# Unicode

##### file load

In [ ]:
import json
import pandas as pd

In [ ]:
data = []
with open("drive/MyDrive/Advertising_By_Personality/data/gpt/ext_gpt_mbti.json", 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))
df = pd.DataFrame(data)

##### unicode

In [ ]:
import swifter
from pshmodule.processing import processing as p

In [ ]:
nlp = p.Nlp()

In [ ]:
df_3000.columns

Index(['mbti', 'marketing_entity', 'marketing_target', 'benefit_conditions',
       'benefits', 'discount_figure', 'promotional_items', 'promotional_place',
       'event_period', 'dow_information', 'season_information',
       'solicitation_point', 'type', 'label'],
      dtype='object')

In [ ]:
df_3000['mbti'] = df_3000.mbti.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['marketing_entity'] = df_3000.marketing_entity.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['marketing_target'] = df_3000.marketing_target.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['benefit_conditions'] = df_3000.benefit_conditions.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['benefits'] = df_3000.benefits.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['discount_figure'] = df_3000.discount_figure.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['promotional_items'] = df_3000.promotional_items.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['promotional_place'] = df_3000.promotional_place.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['event_period'] = df_3000.event_period.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['dow_information'] = df_3000.dow_information.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['season_information'] = df_3000.season_information.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['solicitation_point'] = df_3000.solicitation_point.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))
df_3000['label'] = df_3000.label.swifter.apply(lambda x: nlp.convert_to_other_unicode(x))

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/3000 [00:00<?, ?it/s]

In [ ]:
df_3000.head()

,mbti,marketing_entity,marketing_target,benefit_conditions,benefits,discount_figure,promotional_items,promotional_place,event_period,dow_information,season_information,solicitation_point,type,label
0,이왕이면 재밌게 게임\ud83c\udfae하고 최대 2만P\ud83d\udcb0 받...,없음,없음,매일 놀면서 돈 버기,최대 2만P,없음,게임,없음,없음,없음,없음,없음,NT,꿈은 없고요 그냥 놀고 싶습니다\ud83e\udd38\\이왕이면 재밌게 게임\ud8...
1,"거짓말 같은 할인\ud83e\udd25, 하지만 거짓말이 아닌, 50% 특가⚡, 다...",없음,TAG1,없음,50% 특가,0.5,피노키오전,없음,다가오는 주말,없음,없음,없음,NT,거짓말 같은 할인\ud83e\udd25\\하지만 거짓말이 아닌 [피노키오전] 50%...
2,"오늘은 꼭 빕스 가야 할걸?, [빕스 폭립 원.쁠.원] 이벤트\u2728 하나 가지...",빕스,없음,없음,빕스 폭립 원.쁠.원 이벤트,없음,없음,없음,오늘 종료,없음,없음,빕스,NT,오늘은 꼭 빕스 가야 할걸?\ud83c\udfc3\\오/늘/종/료\u23F0 [빕스...
3,"비스포크가 포인트를 쏜다쏜다쏜다!\ud83d\udd2b, 3만P\ud83d\udcb...",비스포크,없음,"감동 사연 남기기, 응모하기",3만P,없음,비스포크xCJ 0NE 콜라보,없음,없음,없음,없음,없음,NT,비스포크가 포인트를 쏜다쏜다쏜다!\ud83d\udd2b\\[비스포크]에 관한 감동 ...
4,"딱 1천P\ud83d\udcb0만 들고 커몽!, [백악기 공룡의 전설 탐험展] 40...",없음,TAG1,40% 할인권 교환,40% 할인권,0.4,백악기 공룡의 전설 탐험展,없음,없음,없음,없음,없음,NT,딱 1천P\ud83d\udcb0만 들고 커몽!\\[백악기 공룡의 전설 탐험展] 40...


# Save

##### 1차 - 2,000건

In [ ]:
data = []
with open('drive/MyDrive/Advertising_By_Personality/data/backup/train_final_2000.json', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))
df_2000 = pd.DataFrame(data)

mbti 합치기

In [ ]:
df_2000.columns

Index(['NT', 'NF', 'marketing_entity', 'marketing_target',
       'benefit_conditions', 'benefits', 'discount_figure',
       'promotional_items', 'promotional_place', 'event_period',
       'dow_information', 'season_information', 'solicitation_point', 'type',
       'label'],
      dtype='object')

In [ ]:
df_nt = df_2000[df_2000.type == "NT"]
df_nt["mbti"] = df_nt.NT
df_nt = df_nt[['mbti', 'marketing_entity', 'marketing_target', 'benefit_conditions', 'benefits', 'discount_figure', 'promotional_items', 'promotional_place', 'event_period', 'dow_information', 'season_information', 'solicitation_point', 'type', 'label']]

<ipython-input-23-ad801c8a980b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nt["mbti"] = df_nt.NT


In [ ]:
df_nf = df_2000[df_2000.type == "NF"]
df_nf["mbti"] = df_nf.NF
df_nf = df_nf[['mbti', 'marketing_entity', 'marketing_target', 'benefit_conditions', 'benefits', 'discount_figure', 'promotional_items', 'promotional_place', 'event_period', 'dow_information', 'season_information', 'solicitation_point', 'type', 'label']]

<ipython-input-24-2ad9321cb533>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nf["mbti"] = df_nf.NF


In [ ]:
df_2000 = pd.concat([df_nt, df_nf], axis=0)

In [ ]:
df_2000.head()

,mbti,marketing_entity,marketing_target,benefit_conditions,benefits,discount_figure,promotional_items,promotional_place,event_period,dow_information,season_information,solicitation_point,type,label
0,"이 혜택\ud83d\udcb0 놓치지 않을 거예요, [계절밥상] 디너/주말 10,0...",계절밥상,없음,없음,"디너/주말 10,000원, 런치 5,000원 할인","10,000원 디너/주말, 5,000원 런치",계절밥상,없음,9월 30일까지,없음,없음,없음,NT,"이 혜택\ud83d\udcb0 놓치지 않을 거예요\\[계절밥상] 디너/주말 10,0..."
1,"무조건 당첨인데 참여 외않해?\ud83d\udc40, 500P만 있으면 무조건 당첨...",없음,없음,500P,"200만원 상당 여행권, 빕스 쿠폰",없음,없음,없음,없음,없음,없음,없음,NT,무조건 당첨인데 참여 외않해?\ud83d\udc40\\500P만 있으면 무조건 당첨...
2,"아직 손도 안 댄 쿠폰이 있어요!, 미사용 쿠폰 만료까지 D-7\ud83d\udea...",없음,없음,미사용 쿠폰 만료까지 확인,없음,없음,없음,없음,D-7,없음,없음,없음,NT,아직 손도 안 댄 쿠폰이 있어요!\\미사용 쿠폰 만료까지 D-7\ud83d\udea...
3,"주말엔 사라질 그 할인쿠폰!, [0000 1만원 할인쿠폰], 유효기간은 이번 주 일...",없음,없음,할인 쿠폰 사용,1만원 할인,10000원,통돼지구이,없음,이번 주 일요일까지,일요일,없음,없음,NT,주말엔 사라질 그 할인쿠폰!\\[0000 1만원 할인쿠폰] 의 유효기간은 이번 주 ...
4,"득템프 1회 더 찍으면 5,000원 할인 쿠폰도 드린답니다",누리세오,없음,득템프 1회 더 찍기,"5,000원 할인 쿠폰","5,000원",바캉스 위크,없음,없음,없음,없음,없음,NT,"00 바캉스 위크 특가⚡누리세오!\\아참, 득템프 1회 더 찍으면 5,000원 할인..."


2차 - 3,000건

In [ ]:
df = pd.concat([df_2000, df_3000], axis=0)

In [ ]:
df.shape

(5000, 14)

1, 2차 합치기

In [ ]:
temp_dict = [{"mbti": row['mbti'], "marketing_entity": row['marketing_entity'], "marketing_target": row['marketing_target'], "benefit_conditions": row['benefit_conditions'], "benefits": row['benefits'], "discount_figure": row['discount_figure'], "promotional_items": row['promotional_items'], "promotional_place": row['promotional_place'], "event_period": row['event_period'], "dow_information": row['dow_information'], "season_information": row['season_information'], "solicitation_point": row['solicitation_point'], "type": row['type'], "label": row['label']} for _, row in df.iterrows()]
with open("drive/MyDrive/Advertising_By_Personality/data/train_final.json", 'w', encoding='utf-8') as f:
    for line in temp_dict:
        json_record = json.dumps(line, ensure_ascii=False)
        f.write(json_record + '\n')

# ReUnicode

In [ ]:
import json
import pandas as pd
import swifter
from pshmodule.processing import processing as p

In [ ]:
data = []
with open('drive/MyDrive/Advertising_By_Personality/data/train_final.json', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))
df = pd.DataFrame(data)

In [ ]:
nlp = p.Nlp()

In [ ]:
df['mbti'] = df.mbti.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['marketing_entity'] = df.marketing_entity.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['marketing_target'] = df.marketing_target.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['benefit_conditions'] = df.benefit_conditions.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['benefits'] = df.benefits.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['discount_figure'] = df.discount_figure.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['promotional_items'] = df.promotional_items.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['promotional_place'] = df.promotional_place.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['event_period'] = df.event_period.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['dow_information'] = df.dow_information.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['season_information'] = df.season_information.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['solicitation_point'] = df.solicitation_point.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))
df['label'] = df.label.swifter.apply(lambda x: nlp.convert_emojis_in_text(nlp.convert_to_python_unicode(x)))

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
df['mbti'] = df.mbti.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['marketing_entity'] = df.marketing_entity.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['marketing_target'] = df.marketing_target.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['benefit_conditions'] = df.benefit_conditions.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['benefits'] = df.benefits.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['discount_figure'] = df.discount_figure.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['promotional_items'] = df.promotional_items.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['promotional_place'] = df.promotional_place.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['event_period'] = df.event_period.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['dow_information'] = df.dow_information.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['season_information'] = df.season_information.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['solicitation_point'] = df.solicitation_point.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))
df['label'] = df.label.swifter.apply(lambda x: x.encode('utf-8', 'ignore').decode('utf-8'))

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
df.head()

,mbti,marketing_entity,marketing_target,benefit_conditions,benefits,discount_figure,promotional_items,promotional_place,event_period,dow_information,season_information,solicitation_point,type,label
0,"이 혜택💰 놓치지 않을 거예요, [계절밥상] 디너/주말 10,000원, 런치 5,0...",계절밥상,없음,없음,"디너/주말 10,000원, 런치 5,000원 할인","10,000원 디너/주말, 5,000원 런치",계절밥상,없음,9월 30일까지,없음,없음,없음,NT,"이 혜택💰 놓치지 않을 거예요\\[계절밥상] 디너/주말 10,000원, 런치 5,0..."
1,"무조건 당첨인데 참여 외않해?👀, 500P만 있으면 무조건 당첨되는 갓-이벤트✨, ...",없음,없음,500P,"200만원 상당 여행권, 빕스 쿠폰",없음,없음,없음,없음,없음,없음,없음,NT,무조건 당첨인데 참여 외않해?👀\\500P만 있으면 무조건 당첨되는 갓-이벤트✨\2...
2,"아직 손도 안 댄 쿠폰이 있어요!, 미사용 쿠폰 만료까지 D-7🚨 쿠폰함으로 확인하...",없음,없음,미사용 쿠폰 만료까지 확인,없음,없음,없음,없음,D-7,없음,없음,없음,NT,아직 손도 안 댄 쿠폰이 있어요!\\미사용 쿠폰 만료까지 D-7🚨 쿠폰함으로 확인하...
3,"주말엔 사라질 그 할인쿠폰!, [0000 1만원 할인쿠폰], 유효기간은 이번 주 일...",없음,없음,할인 쿠폰 사용,1만원 할인,10000원,통돼지구이,없음,이번 주 일요일까지,일요일,없음,없음,NT,주말엔 사라질 그 할인쿠폰!\\[0000 1만원 할인쿠폰] 의 유효기간은 이번 주 ...
4,"득템프 1회 더 찍으면 5,000원 할인 쿠폰도 드린답니다",누리세오,없음,득템프 1회 더 찍기,"5,000원 할인 쿠폰","5,000원",바캉스 위크,없음,없음,없음,없음,없음,NT,"00 바캉스 위크 특가⚡누리세오!\\아참, 득템프 1회 더 찍으면 5,000원 할인..."


In [ ]:
temp_dict = [{"mbti": row['mbti'], "marketing_entity": row['marketing_entity'], "marketing_target": row['marketing_target'], "benefit_conditions": row['benefit_conditions'], "benefits": row['benefits'], "discount_figure": row['discount_figure'], "promotional_items": row['promotional_items'], "promotional_place": row['promotional_place'], "event_period": row['event_period'], "dow_information": row['dow_information'], "season_information": row['season_information'], "solicitation_point": row['solicitation_point'], "type": row['type'], "label": row['label']} for _, row in df.iterrows()]
with open("drive/MyDrive/Advertising_By_Personality/data/gpt/for_selling.json", 'w', encoding='utf-8') as f:
    for line in temp_dict:
        json_record = json.dumps(line, ensure_ascii=False)
        f.write(json_record + '\n')

# Transform Save

In [ ]:
import json
import pandas as pd

In [ ]:
!ls drive/MyDrive/Advertising_By_Personality/data/reports/

predict.xlsx  모델결과.gsheet
test.xlsx     성환님요청사항_TF특징_학습용데이터.xlsx


In [ ]:
df = pd.read_excel('drive/MyDrive/Advertising_By_Personality/data/reports/모델 결과.xlsx', sheet_name="chatgpt 소구점 추출 - 2차")

In [ ]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,gpt3.5-turbo,Unnamed: 4,gpt4,Unnamed: 6
0,no,구분,CJ 광고 문구,소구점,소구점이 반영된 광고 문구,소구점,소구점이 반영된 광고 문구
1,1,NT,"이 혜택💰 놓치지 않을 거예요\\[계절밥상] 디너/주말 10,000원, 런치 5,0...","혜택 마감 임박 강조, 제품 구매 시 얻을 수 있는 실질적이고 금전적인 혜택을 강조...",NaN,NaN,NaN
2,2,NT,무조건 당첨인데 참여 외않해?👀\\500P만 있으면 무조건 당첨되는 갓-이벤트✨\2...,"작은 노력으로 비교적 큰 이득을 받을 수 있음을 강조, 실질적이고 금전적인 혜택 강조",NaN,NaN,NaN
3,3,NT,아직 손도 안 댄 쿠폰이 있어요!\\미사용 쿠폰 만료까지 D-7🚨 쿠폰함으로 확인하...,"혜택 마감 임박 강조, 고객에게 말을 건네듯이, 의미를 부각시킬 수 있는 의성어 사용",NaN,NaN,NaN
4,4,NT,주말엔 사라질 그 할인쿠폰!\\[0000 1만원 할인쿠폰] 의 유효기간은 이번 주 ...,"혜택 마감 임박 강조, 심상을 부각시킬 수 있는 의성어나 의태어 사용, 쿠폰 이용 ...",NaN,NaN,NaN


In [ ]:
df.columns = df.iloc[0]
df = df.iloc[1:]

In [ ]:
df.head()

,no,구분,CJ 광고 문구,소구점,소구점이 반영된 광고 문구,소구점,소구점이 반영된 광고 문구
1,1,NT,"이 혜택💰 놓치지 않을 거예요\\[계절밥상] 디너/주말 10,000원, 런치 5,0...","혜택 마감 임박 강조, 제품 구매 시 얻을 수 있는 실질적이고 금전적인 혜택을 강조...",NaN,NaN,NaN
2,2,NT,무조건 당첨인데 참여 외않해?👀\\500P만 있으면 무조건 당첨되는 갓-이벤트✨\2...,"작은 노력으로 비교적 큰 이득을 받을 수 있음을 강조, 실질적이고 금전적인 혜택 강조",NaN,NaN,NaN
3,3,NT,아직 손도 안 댄 쿠폰이 있어요!\\미사용 쿠폰 만료까지 D-7🚨 쿠폰함으로 확인하...,"혜택 마감 임박 강조, 고객에게 말을 건네듯이, 의미를 부각시킬 수 있는 의성어 사용",NaN,NaN,NaN
4,4,NT,주말엔 사라질 그 할인쿠폰!\\[0000 1만원 할인쿠폰] 의 유효기간은 이번 주 ...,"혜택 마감 임박 강조, 심상을 부각시킬 수 있는 의성어나 의태어 사용, 쿠폰 이용 ...",NaN,NaN,NaN
5,5,NT,"00 바캉스 위크 특가⚡누리세오!\\아참, 득템프 1회 더 찍으면 5,000원 할인...","눈길을 끌기 위해 유명한 노래 가사나 영화 대사 등을 인용, 시즌 정보에 맞게 예상...",NaN,NaN,NaN


In [ ]:
temp_dict = [{"type": row['구분'], "origin": row['CJ 광고 문구'], "selling_point": row['소구점']} for _, row in df.iterrows()]
with open("drive/MyDrive/Advertising_By_Personality/data/gpt/temp.json", 'w', encoding='utf-8') as f:
    for line in temp_dict:
        json_record = json.dumps(line, ensure_ascii=False)
        f.write(json_record + '\n')

TypeError: ignored

# Count Check

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction']


In [ ]:
import json
import config as cfg
import pandas as pd
from pshmodule.utils import filemanager as fm

In [ ]:
data = []
with open(cfg.for_selling_ntnf, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))
df = pd.DataFrame(data)

In [ ]:
fm.save('/content/drive/MyDrive/Advertising_By_Personality/data/reports/check.xlsx', df)

Saved 5000 records


# Selling_Point Check

##### 1,000건

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction']


In [ ]:
import json
import config as cfg
import pandas as pd
from pshmodule.utils import filemanager as fm

In [ ]:
data = []
with open(cfg.EXT_GPT_SELL, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))
df = pd.DataFrame(data)

In [ ]:
df.columns

Index(['marketing_entity', 'marketing_target', 'benefit_conditions',
       'benefits', 'discount_figure', 'promotional_items', 'promotional_place',
       'event_period', 'dow_information', 'season_information', 'type',
       'label', 'use_market', 'sell_point', 'sell_title', 'sell_content'],
      dtype='object')

In [ ]:
df = df.iloc[:1000]

In [ ]:
df.shape

(1000, 16)

In [ ]:
df["advertise_sent"] = df["sell_title"] + "////" + df["sell_content"]

In [ ]:
df = df[['type', 'label', 'use_market', 'sell_point', 'advertise_sent']]

In [ ]:
df.head()

,type,label,use_market,sell_point,advertise_sent
0,NT,"이 혜택💰 놓치지 않을 거예요\\[계절밥상] 디너/주말 10,000원, 런치 5,0...","마케팅 주체 : 계절밥상, 혜택 : 디너/주말 10,000원, 런치 5,000원 할인","혜택 마감 임박 강조, 시즌 정보를 활용하여 고객의 니즈에 공감대 형성하기, 할인 ...",디너/주말 할인!! 오늘 끝나요🚨////지금이 딱 기회! [계절밥상]에서 디너/주말...
1,NT,무조건 당첨인데 참여 외않해?👀\\500P만 있으면 무조건 당첨되는 갓-이벤트✨\2...,"마케팅 주체 : 없음, 혜택 : 200만원 상당 여행권, 빕스 쿠폰","고객이 특별한 존재라는 느낌이 들게 강조, 아는 사람들 사이에서는 굉장히 화제되고 ...",참여 안 해? 후회 함부로는 못 해요👀////지금 500P만 있으면 무조건 당첨되는...
2,NT,아직 손도 안 댄 쿠폰이 있어요!\\미사용 쿠폰 만료까지 D-7🚨 쿠폰함으로 확인하...,"마케팅 주체 : 없음, 혜택 : 없음","혜택 마감 임박을 강조하여 희소성 알리기, 고객에게 말을 건네듯이, 의미를 부각시킬...",쿠폰: 저.. 여기 있어요🥲////혹쉬.. 잊으셨나요?👀 미사용 쿠폰 확인하러 쿠폰...
3,NT,주말엔 사라질 그 할인쿠폰!\\[0000 1만원 할인쿠폰] 의 유효기간은 이번 주 ...,"마케팅 주체 : 없음, 혜택 : 1만원 할인","혜택 마감 임박 강조, 맛있는 음식을 강조, 고객에게 말을 건네듯이, 의미를 부각시...",마감임박! 미친 가격 1만원 할인🔥////[0000 1만원 할인쿠폰]의 유효기간은 ...
4,NT,"00 바캉스 위크 특가⚡누리세오!\\아참, 득템프 1회 더 찍으면 5,000원 할인...","마케팅 주체 : 누리세오, 혜택 : 5,000원 할인 쿠폰","올여름 바캉스 시즌에 맞게 고객의 니즈에 공감대 형성하여 강조, 작은 노력으로 비교...","더 싼 가격으로 행복한 바캉스 연출하기⚡////드디어 올여름 바캉스 시즌! 👙 5,..."


In [ ]:
fm.save('/content/drive/MyDrive/Advertising_By_Personality/data/reports/check.xlsx', df)

Saved 1000 records


##### 데이터 정리

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction']


In [ ]:
import json
import config as cfg
import pandas as pd
from pshmodule.utils import filemanager as fm

In [ ]:
data = []
with open(cfg.EXT_GPT_SELL, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))
df = pd.DataFrame(data)

In [ ]:
df = df.iloc[:1000]

In [ ]:
df.label.iloc[999]

'주변에 4~8세 아이가 있다구요❓\\[윙크] 무료학습 신청하면 5,000P�랑 책 3권�을 드립니다! 완전 핵이득 혜택✨ 바로 알려주러 가실 거죠~?'

In [ ]:
df.loc[999, 'label'] = "주변에 4~8세 아이가 있다구요❓\\\\[윙크] 무료학습 신청하면 5,000P랑 책 3권을 드립니다! 완전 핵이득 혜택✨ 바로 알려주러 가실 거죠~?"

In [ ]:
df.tail()

,marketing_entity,marketing_target,benefit_conditions,benefits,discount_figure,promotional_items,promotional_place,event_period,dow_information,season_information,type,label,use_market,sell_point,sell_title,sell_content
995,2020 MAMA,없음,투표 후 인증하기,한정판 기프트카드 1만원권 증정,없음,레어 깊카,앱,없음,없음,없음,NT,다른 건 참아도 한정판은 못 참아💢\\[2020 MAMA] 투표 후 인증하면✔️ 한...,"마케팅 주체 : 2020 MAMA, 혜택 : 한정판 기프트카드 1만원권 증정","고객이 투표 후 인증하면 얻을 수 있는 보상을 암시적으로 나타내기, 한정판임을 강조...",오늘이 기회야! 한정판 기프트카드 1만원권 증정💵,MAMA 투표 후 인증하면✔️ 레어 깊카 챙기세요💌 앱에서 터치 몇 번만 하면 한정...
996,없음,없음,손가락 하나 클릭,"대치동 교재, 1:4 온라인 초등수학 50분, 3000P",없음,"대치동 교재, 온라인 초등수학",없음,없음,없음,없음,NT,공부왕 찐천재 되러 가실?🎓\\✔️대치동 교재 ✔️1:4 온라인 초등수학 50분 ✔...,"마케팅 주체 : 없음, 혜택 : 대치동 교재, 1:4 온라인 초등수학 50분, 3000P","고객에게 공부하는 경험을 미래적으로 연결하고, 해당 프로모션의 혜택을 강조하기",공부왕 되러 가실?🎓,무적의 대치동 교재와 1:4 온라인 초등수학 50분🔢그리고 3000P 몽땅 드리는 ...
997,국립현대미술관 덕수궁,없음,없음,"최소 비용 , 최대 효율",없음,"야외 전시, 낙엽 밟기",국립현대미술관 덕수궁,없음,없음,가을,NT,가을엔 낙엽 한번 밟아줘야죠🍂\\일석이조로 가을 즐기는 법! [국립현대미술관 덕수궁...,"마케팅 주체 : 국립현대미술관 덕수궁, 혜택 : 최소 비용 , 최대 효율",고객이 가을을 즐기는 방법과 연결하여 별도의 비용이 들지 않고도 쉽게 즐길 수 있는...,가을 낙엽 한번 밟아줘야지🍂,가을을 즐기는 최소 비용+최대 효율! [국립현대미술관 덕수궁]에서 야외 전시도 보고...
998,LG,없음,게임 참여,LG시네빔,없음,없음,없음,없음,없음,없음,NT,30초 만에 LG시네빔 받기✨\\재밌게 게임도 하고 LG시네빔🎁도 받고! 완전 핵이...,"마케팅 주체 : LG, 혜택 : LG시네빔","고객이 작은 노력으로 비교적 큰 이득을 받을 수 있음을 강조, 특정 상품이나 선물을...",오늘까지만! LG시네빔 30초 만에 받아가세요✨,재밌게 게임도 하고 LG시네빔🎁도 받고! 완전 핵이득이잖아~❣️ [TAG1]님도 두...
999,없음,4~8세 아이,무료학습 신청,"5000P, 책 3권",없음,없음,없음,없음,없음,없음,NT,"주변에 4~8세 아이가 있다구요❓\\[윙크] 무료학습 신청하면 5,000P랑 책 3...","마케팅 주체 : 없음, 혜택 : 5000P, 책 3권","고객의 작은 노력으로 비교적 큰 이득을 받을 수 있음을 강조, 고객의 니즈를 공감대...",주변에 4~8세 아이가 있다면❗️,"[윙크] 무료학습 신청하면 5,000P와 책 3권 드려요! 완전 핵이득 혜택✨ 바로..."


In [ ]:
temp_dict = [
              {
                  "marketing_entity": row["marketing_entity"],
                  "marketing_target": row["marketing_target"],
                  "benefit_conditions": row["benefit_conditions"],
                  "benefits": row["benefits"],
                  "discount_figure": row["discount_figure"],
                  "promotional_items": row["promotional_items"],
                  "promotional_place": row["promotional_place"],
                  "event_period": row["event_period"],
                  "dow_information": row["dow_information"],
                  "season_information": row["season_information"],
                  "type": row["type"],
                  "label": row["label"].strip(),
                  "use_market": row["use_market"],
                  "sell_point": row["sell_point"],
                  "sell_title": row["sell_title"],
                  "sell_content": row["sell_content"],
              }
              for _, row in df.iterrows()
          ]


with open("drive/MyDrive/Advertising_By_Personality/data/gpt/ext_gpt_sell2.json", 'w', encoding='utf-8') as f:
    for line in temp_dict:
        json_record = json.dumps(line, ensure_ascii=False)
        f.write(json_record + '\n')

# Data Save for train

##### 2,500건


In [6]:
import sys
sys.path.append('/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Advertising_By_Personality/src/business_model/keyword_extraction']


In [15]:
import json
import swifter
import config as cfg
import pandas as pd
from pshmodule.utils import filemanager as fm
from pshmodule.processing import processing as p

In [8]:
data = []
with open(cfg.EXT_GPT_SELL, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))
df = pd.DataFrame(data)

In [9]:
col_list = list(df.columns)

In [10]:
col_list.remove('label')
col_list.remove('use_market')

In [11]:
df_temp = df[col_list]

In [12]:
df_temp.columns

Index(['marketing_entity', 'marketing_target', 'benefit_conditions',
       'benefits', 'discount_figure', 'promotional_items', 'promotional_place',
       'event_period', 'dow_information', 'season_information', 'type',
       'sell_point', 'sell_title', 'sell_content'],
      dtype='object')

In [13]:
df_temp = df_temp.iloc[:2500]

In [14]:
df_temp.shape

(2500, 14)

convert to other unicode

In [16]:
nlp = p.Nlp()

In [17]:
df_temp["label"] = df_temp["sell_title"] + "\\\\" + df_temp["sell_content"]

In [20]:
df_temp["label_unicode"] = df_temp.label.swifter.apply(nlp.convert_to_other_unicode)

Pandas Apply:   0%|          | 0/2500 [00:00<?, ?it/s]

In [21]:
df_temp.head()

,marketing_entity,marketing_target,benefit_conditions,benefits,discount_figure,promotional_items,promotional_place,event_period,dow_information,season_information,type,sell_point,sell_title,sell_content,label,label_unicode
0,계절밥상,없음,없음,"디너/주말 10,000원, 런치 5,000원 할인","10,000원 디너/주말, 5,000원 런치",계절밥상,없음,9월 30일까지,없음,없음,NT,"혜택 마감 임박 강조, 시즌 정보를 활용하여 고객의 니즈에 공감대 형성하기, 할인 ...",디너/주말 할인!! 오늘 끝나요🚨,"지금이 딱 기회! [계절밥상]에서 디너/주말 맛있는 식사할 때가 왔어요! 10,00...",디너/주말 할인!! 오늘 끝나요🚨\\지금이 딱 기회! [계절밥상]에서 디너/주말 맛...,디너/주말 할인!! 오늘 끝나요\ud83d\udea8\\지금이 딱 기회! [계절밥상...
1,없음,없음,500P,"200만원 상당 여행권, 빕스 쿠폰",없음,없음,없음,없음,없음,없음,NT,"고객이 특별한 존재라는 느낌이 들게 강조, 아는 사람들 사이에서는 굉장히 화제되고 ...",참여 안 해? 후회 함부로는 못 해요👀,지금 500P만 있으면 무조건 당첨되는 인기 끝판왕 이벤트✨ 여행권부터 빕스 쿠폰까...,참여 안 해? 후회 함부로는 못 해요👀\\지금 500P만 있으면 무조건 당첨되는 인...,참여 안 해? 후회 함부로는 못 해요\ud83d\udc40\\지금 500P만 있으면...
2,없음,없음,미사용 쿠폰 만료까지 확인,없음,없음,없음,없음,D-7,없음,없음,NT,"혜택 마감 임박을 강조하여 희소성 알리기, 고객에게 말을 건네듯이, 의미를 부각시킬...",쿠폰: 저.. 여기 있어요🥲,혹쉬.. 잊으셨나요?👀 미사용 쿠폰 확인하러 쿠폰함으로 고고👉,쿠폰: 저.. 여기 있어요🥲\\혹쉬.. 잊으셨나요?👀 미사용 쿠폰 확인하러 쿠폰함으...,쿠폰: 저.. 여기 있어요\ud83e\udd72\\혹쉬.. 잊으셨나요?\ud83d\...
3,없음,없음,할인 쿠폰 사용,1만원 할인,10000원,통돼지구이,없음,이번 주 일요일까지,일요일,없음,NT,"혜택 마감 임박 강조, 맛있는 음식을 강조, 고객에게 말을 건네듯이, 의미를 부각시...",마감임박! 미친 가격 1만원 할인🔥,[0000 1만원 할인쿠폰]의 유효기간은 이번 주 일요일까지🚨 쫀득한 통돼지구이를 ...,마감임박! 미친 가격 1만원 할인🔥\\[0000 1만원 할인쿠폰]의 유효기간은 이번...,마감임박! 미친 가격 1만원 할인\ud83d\udd25\\[0000 1만원 할인쿠폰...
4,누리세오,없음,득템프 1회 더 찍기,"5,000원 할인 쿠폰","5,000원",바캉스 위크,없음,없음,없음,없음,NT,"올여름 바캉스 시즌에 맞게 고객의 니즈에 공감대 형성하여 강조, 작은 노력으로 비교...",더 싼 가격으로 행복한 바캉스 연출하기⚡,"드디어 올여름 바캉스 시즌! 👙 5,000원 할인 쿠폰 받을 시 득템프 1회 더 찍...","더 싼 가격으로 행복한 바캉스 연출하기⚡\\드디어 올여름 바캉스 시즌! 👙 5,00...",더 싼 가격으로 행복한 바캉스 연출하기⚡\\드디어 올여름 바캉스 시즌! \ud83d...


Save

In [22]:
temp_dict = [
              {
                  "marketing_entity": row["marketing_entity"],
                  "marketing_target": row["marketing_target"],
                  "benefit_conditions": row["benefit_conditions"],
                  "benefits": row["benefits"],
                  "discount_figure": row["discount_figure"],
                  "promotional_items": row["promotional_items"],
                  "promotional_place": row["promotional_place"],
                  "event_period": row["event_period"],
                  "dow_information": row["dow_information"],
                  "season_information": row["season_information"],
                  "sell_point": row["sell_point"],
                  "label": row["label_unicode"]
              }
              for _, row in df_temp.iterrows()
          ]


with open("drive/MyDrive/Advertising_By_Personality/data/train_t5_selling.json", 'w', encoding='utf-8') as f:
    for line in temp_dict:
        json_record = json.dumps(line, ensure_ascii=False)
        f.write(json_record + '\n')